In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

Preprocessing producao, and comercializacao

In [2]:
# Read the data from the CSV file, as one is needed because they follow the same format

producao_data = pd.read_csv('raw_data/Producao.csv', sep=';')
producao_data.head()

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556


In [3]:
comercio_data = pd.read_csv('raw_data/Comercio.csv', sep=';')
comercio_data.head()

,id,control,Produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310


In [4]:
categories_producao = producao_data['control'][producao_data['control'].str.isupper()].reset_index(drop=True).values

categories_producao_df = pd.DataFrame(
    columns=['category'],
    data=categories_producao
)

In [5]:
comercio_data.rename(columns={'Produto': 'produto'}, inplace=True)
comercio_data['control'] = comercio_data['control'].fillna(comercio_data['produto'])


In [6]:
categories_comercio = comercio_data['control'][comercio_data['control'].str.isupper()].reset_index(drop=True).values

categories_comercio_df = pd.DataFrame(
    columns=['category'],
    data=categories_comercio
)


In [7]:
category_df = pd.concat([categories_producao_df, categories_comercio_df], ignore_index=True)
category_df.drop_duplicates(inplace=True)
category_df.reset_index(drop=True, inplace=True)
category_df

,category
0,VINHO DE MESA
1,VINHO FINO DE MESA (VINIFERA)
2,SUCO
3,DERIVADOS
4,VINHO FINO DE MESA
5,VINHO FRIZANTE
6,VINHO ORGÂNICO
7,VINHO ESPECIAL
8,ESPUMANTES
9,SUCO DE UVAS


In [8]:
category_df['category_id'] = category_df.index

In [9]:
category_df

,category,category_id
0,VINHO DE MESA,0
1,VINHO FINO DE MESA (VINIFERA),1
2,SUCO,2
3,DERIVADOS,3
4,VINHO FINO DE MESA,4
5,VINHO FRIZANTE,5
6,VINHO ORGÂNICO,6
7,VINHO ESPECIAL,7
8,ESPUMANTES,8
9,SUCO DE UVAS,9


In [10]:
def categorize_data(producao_data):
    current_category = None
    categories = []

    # Itera sobre as linhas do DataFrame
    for index, row in producao_data.iterrows():
        if row['control'].isupper():
            # Atualiza a categoria atual
            current_category = row['control']
        categories.append(current_category)

    return categories

In [11]:
def filter_data(producao_data, subcategory_column='produto'):
    filtered_data = producao_data[
        ~(producao_data['control'].str.isupper() & producao_data[subcategory_column].str.isupper())]
    return filtered_data

In [12]:
def map_categories_to_ids(producao_data, category_df, category_column='category', id_column='category_id',
                          control_column='control'):
    category_to_id = dict(zip(category_df[category_column], category_df[id_column]))
    producao_data[control_column] = producao_data[control_column].map(category_to_id)
    return producao_data


def map_subcategories_to_ids(data, subcategories, subcategory_column='subcategory', id_column='subcategory_id'):
    subcategories_to_id = dict(zip(subcategories[subcategory_column], subcategories[id_column]))
    data.rename(columns={subcategory_column: id_column}, inplace=True)
    data[id_column] = data[id_column].map(subcategories_to_id)
    return data

In [13]:
category_df

,category,category_id
0,VINHO DE MESA,0
1,VINHO FINO DE MESA (VINIFERA),1
2,SUCO,2
3,DERIVADOS,3
4,VINHO FINO DE MESA,4
5,VINHO FRIZANTE,5
6,VINHO ORGÂNICO,6
7,VINHO ESPECIAL,7
8,ESPUMANTES,8
9,SUCO DE UVAS,9


In [14]:
only_categories_producao = producao_data[producao_data['control'].str.isupper()]
only_categories_producao.rename(columns={'control': 'category'}, inplace=True)
only_categories_producao.drop(columns=['id', 'produto'], inplace=True)
only_categories_producao = only_categories_producao.melt(id_vars=['category'], var_name='year', value_name='value')
only_categories_producao['year'] = only_categories_producao['year'].astype(int)


In [15]:
only_categories_comercio = comercio_data[comercio_data['control'].str.isupper()]
only_categories_comercio.rename(columns={'control': 'category'}, inplace=True)
only_categories_comercio.drop(columns=['id', 'produto'], inplace=True)
only_categories_comercio = only_categories_comercio.melt(id_vars=['category'], var_name='year', value_name='value')
only_categories_comercio['year'] = only_categories_comercio['year'].astype(int)


In [16]:
producao_data

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556
5,6,vv_Tinto,Tinto,7591557,7265666,6782837,3419625,10047658,14731106,12405154,...,17208996,16745896,8774847,21442212,19118254,17389377,15451883,20433249,24417918,23615783
6,7,vv_Branco,Branco,15562889,15655709,13289304,7930070,18927471,21933695,19874659,...,20054804,19561966,8705066,21928400,18297257,18193055,15487915,20867999,20896613,20693437
7,8,vv_Rosado,Rosado,744900,664687,1006630,1018715,2668995,2759789,2220777,...,1200514,841120,590713,1167258,1291709,2032990,1576888,2173750,2197265,1959336
8,9,SUCO,SUCO,1097771,2296654,3509440,0,0,0,0,...,76723537,87894468,42210389,77161971,65467906,77805352,69261287,100932264,65809079,67045238
9,10,su_Suco de uva simples,Suco de uva integral,1097771,2296654,3509440,0,0,0,0,...,43331223,52233155,31117869,46865626,34367996,50239767,40718523,68038479,35248305,38122173


In [17]:
producao_data['control'] = categorize_data(producao_data)
producao_data = filter_data(producao_data)
producao_data = map_categories_to_ids(producao_data, category_df)
producao_data.rename(columns={'control': 'category_id'}, inplace=True)
producao_data.rename(columns={'produto': 'subcategory'}, inplace=True)
producao_data.drop(columns=['id'], inplace=True)
producao_data = producao_data.melt(id_vars=['category_id', 'subcategory'], var_name='year', value_name='value')
producao_data['year'] = producao_data['year'].astype(int)


In [18]:
products_to_lowercase = ['VINHO FRIZANTE', 'VINHO ORGÂNICO', 'SUCO DE UVAS CONCENTRADO']


def fix_products_names(comercio_data):
    for product in products_to_lowercase:
        comercio_data['produto'] = comercio_data['produto'].str.replace(product, product.capitalize())
        comercio_data['produto'] = comercio_data['produto'].str.strip()
    return comercio_data


comercio_data = fix_products_names(comercio_data)

In [19]:
comercio_data

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,ou_Vinho_base,Vinho base para espumantes,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,59,ou_Vinho_composto,Vinho composto,3386173,3645774,3921573,4168841,3670736,3493104,5429800,...,450121,413735,377685,485944,249165,366549,63625,227116,32000,981
59,60,ou_Vinho_licoroso,Vinho licoroso,3013056,2475632,2407816,2443944,1921867,2205036,2474364,...,616904,510975,546189,496746,491165,465964,362340,376597,385006,421974
60,61,ou_Vinho_leve,Vinho leve,0,0,0,0,0,0,0,...,0,0,0,0,198,306,396,0,27,132064


In [20]:
comercio_data['control'] = categorize_data(comercio_data)
comercio_data = filter_data(comercio_data)
comercio_data = map_categories_to_ids(comercio_data, category_df)
comercio_data.rename(columns={'control': 'category_id'}, inplace=True)
comercio_data.rename(columns={'produto': 'subcategory'}, inplace=True)
comercio_data.drop(columns=['id'], inplace=True)
comercio_data = comercio_data.melt(id_vars=['category_id', 'subcategory'], var_name='year', value_name='value')
comercio_data['year'] = comercio_data['year'].astype(int)


In [21]:
subcategories_producao = producao_data.loc[:, ['category_id', 'subcategory']].drop_duplicates().reset_index(drop=True)
subcategories_producao

,category_id,subcategory
0,0,Tinto
1,0,Branco
2,0,Rosado
3,1,Tinto
4,1,Branco
5,1,Rosado
6,2,Suco de uva integral
7,2,Suco de uva concentrado
8,2,Suco de uva adoçado
9,2,Suco de uva orgânico


In [22]:
subcategories_comercio = comercio_data.loc[:, ['category_id', 'subcategory']].drop_duplicates().reset_index(drop=True)
subcategories_comercio

,category_id,subcategory
0,0,Tinto
1,0,Rosado
2,0,Branco
3,4,Tinto
4,4,Rosado
5,4,Branco
6,5,Vinho frizante
7,6,Vinho orgânico
8,7,Tinto
9,7,Rosado


In [23]:
# Merge the subcategories from both datasets
subcategories = pd.concat([subcategories_producao, subcategories_comercio], ignore_index=True)
subcategories.drop_duplicates(inplace=True)
subcategories.reset_index(drop=True, inplace=True)
subcategories['subcategory_id'] = subcategories.index
subcategories

,category_id,subcategory,subcategory_id
0,0,Tinto,0
1,0,Branco,1
2,0,Rosado,2
3,1,Tinto,3
4,1,Branco,4
...,...,...,...
95,11,Vinho base para espumantes,95
96,11,Vinho composto,96
97,11,Vinho licoroso,97
98,11,Vinho leve,98


In [24]:
producao_data_subcategory = producao_data.drop(columns=['category_id'])
producao_data_subcategory = map_subcategories_to_ids(producao_data_subcategory, subcategories)


In [25]:
comercio_data_subcategory = comercio_data.drop(columns=['category_id'])
comercio_data_subcategory = map_subcategories_to_ids(comercio_data_subcategory, subcategories)


Preprocessing the processamento related datasets -> grape varieties

In [26]:
#loading json file

import json

with open('raw_data/opt_03_sub_buttons.json') as f:
    data = json.load(f)

In [27]:
#creating a dataframe with the grape varieties
grape_varieties = [button['text'] for button in data['sub_buttons']]
grapes_varieties_df = pd.DataFrame(
    columns=['grapes_variety'],
    data=grape_varieties
)


In [28]:
viniferas_table = pd.read_csv('raw_data/ProcessaViniferas.csv', sep=';')
viniferas_table['variety_id'] = \
grapes_varieties_df[grapes_varieties_df['grapes_variety'].str.contains('Viníferas')].index[0]
viniferas_table.head()

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,variety_id
0,1,TINTAS,TINTAS,10448228,11012833,10798824,8213674,17457849,22593885,20265190,...,29935627,13370866,32850915,26868514,nd,28003505,93296587,*,"35881118,23",0
1,2,ti_Alicante Bouschet,Alicante Bouschet,0,0,0,0,0,0,0,...,1519576,908841,2040198,2103844,nd,2272985,811140,*,"4108858,21",0
2,3,ti_Ancelota,Ancelota,0,0,0,0,0,0,0,...,773526,179028,733907,492106,nd,481402,6513974,*,"783688,39",0
3,4,ti_Aramon,Aramon,0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,0,0
4,5,ti_Alfrocheiro,Alfrocheiro,0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,0,0


In [29]:
viniferas_table['control'].replace('BRANCASEROSADAS', 'BRANCAS E ROSADAS', inplace=True)
viniferas_grape_categories_df = viniferas_table.loc[:, ['control', 'variety_id']][
    viniferas_table['control'].str.isupper()].reset_index(drop=True)


In [30]:
americanas_table = pd.read_csv('raw_data/ProcessaAmericanas.csv', sep='\\t', engine='python')
americanas_table['variety_id'] = \
grapes_varieties_df[grapes_varieties_df['grapes_variety'].str.contains('Americanas')].index[0]
americanas_table.head()

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,variety_id
0,1,TINTAS,TINTAS,284285642,193695427,187251954,161593877,295404693,282003228,222289729,...,537592720,235356261,600605876,519506589,nd,376136313,394224543,*,502666358,1
1,2,ti_Bacarina,Bacarina,82899,106962,67464,58690,138158,101454,57297,...,0,0,0,0,nd,0,0,*,0,1
2,3,ti_Bailey,Bailey,0,0,0,0,0,0,0,...,963159,442784,1370092,539742,nd,534981,4092669,*,587066,1
3,4,ti_Bordo,Bordo,7242197,7227090,6530686,5584243,13341412,16023998,12725233,...,137467196,60976531,160146475,158405972,nd,129978861,117655879,*,154310837,1
4,5,ti_Bourdin (S),Bourdin (S),0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,0,1


In [31]:
americanas_table['control'].replace('BRANCASEROSADAS', 'BRANCAS E ROSADAS', inplace=True)
americanas_table_categories_df = americanas_table.loc[:, ['control', 'variety_id']][
    americanas_table['control'].str.isupper()].reset_index(drop=True)


In [32]:
mesa_table = pd.read_csv('raw_data/ProcessaMesa.csv', sep='\\t', engine='python')
mesa_table['variety_id'] = grapes_varieties_df[grapes_varieties_df['grapes_variety'] == 'Uvas de mesa'].index[0]
mesa_table.head()

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,variety_id
0,1,TINTAS,TINTAS,56976,43390,4428,8939,125563,183731,88512,...,108797,51310,85510,62567,nd,63474,21732,*,175030,2
1,2,ti_ Alphonse Lavallee,Alphonse Lavallee,31878,2333,170,7690,124762,74293,23684,...,0,0,0,0,nd,0,0,*,0,2
2,3,ti_ Moscato de Hamburgo,Moscato de Hamburgo,25098,41057,4258,1249,801,109438,64828,...,108797,51310,85510,62567,nd,63474,21732,*,175030,2
3,4,BRANCAS,BRANCAS,3900,16335,6829,8052,278951,587745,153445,...,64540,3260,5900,730,nd,45610,52760,*,13440,2
4,5,br_Cardinal,Cardinal,0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,0,2


In [33]:
mesa_table_categories_df = mesa_table.loc[:, ['control', 'variety_id']][
    mesa_table['control'].str.isupper()].reset_index(drop=True)


In [34]:
sem_classed_table = pd.read_csv('raw_data/ProcessaSemClass.csv', sep='\\t', engine='python')
sem_classed_table['variety_id'] = \
grapes_varieties_df[grapes_varieties_df['grapes_variety'] == 'Sem classificação'].index[0]
sem_classed_table.head()

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,variety_id
0,1,sc,Sem classificação,3675463,665425,197232,491357,57307,540146,24440,...,0,0,0,0,nd,166947,0,*,0,3


In [35]:
sem_classed_table_categories_df = sem_classed_table.loc[:, ['control', 'variety_id']]


In [36]:
grape_categories = pd.concat([viniferas_grape_categories_df, americanas_table_categories_df, mesa_table_categories_df,
                              sem_classed_table_categories_df], ignore_index=True)
grape_categories.rename(columns={'control': 'grapes_categories'}, inplace=True)
grape_categories.drop_duplicates(inplace=True)
grape_categories.reset_index(drop=True, inplace=True)
grape_categories

,grapes_categories,variety_id
0,TINTAS,0
1,BRANCAS E ROSADAS,0
2,TINTAS,1
3,BRANCAS E ROSADAS,1
4,TINTAS,2
5,BRANCAS,2
6,sc,3


In [37]:
viniferas_table

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,variety_id
0,1,TINTAS,TINTAS,10448228,11012833,10798824,8213674,17457849,22593885,20265190,...,29935627,13370866,32850915,26868514,nd,28003505,93296587,*,"35881118,23",0
1,2,ti_Alicante Bouschet,Alicante Bouschet,0,0,0,0,0,0,0,...,1519576,908841,2040198,2103844,nd,2272985,811140,*,"4108858,21",0
2,3,ti_Ancelota,Ancelota,0,0,0,0,0,0,0,...,773526,179028,733907,492106,nd,481402,6513974,*,"783688,39",0
3,4,ti_Aramon,Aramon,0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,0,0
4,5,ti_Alfrocheiro,Alfrocheiro,0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,135,br_Vermentino,Vermentino,20461,6097,8945,6153,8954,7070,6478,...,790,0,0,0,nd,0,0,*,28218,0
135,136,br_Vernaccia,Vernaccia,122467,190297,111691,65995,114553,135975,142787,...,4760,0,5100,4020,nd,95680,0,*,800,0
136,137,br_Viogner,Viogner,0,0,0,0,0,0,0,...,0,0,0,0,nd,0,0,*,"21794,25",0
137,138,br_Viognier,Viognier,0,0,0,0,0,0,0,...,285217,95231,365103,470555,nd,531026,78305,*,"974004,64",0


In [38]:
viniferas_table['control'] = categorize_data(viniferas_table)
viniferas_table = filter_data(viniferas_table, subcategory_column='cultivar')
viniferas_table = map_categories_to_ids(viniferas_table, grape_categories, category_column='grapes_categories',
                                        control_column='control', id_column='variety_id')
viniferas_table.rename(columns={'control': 'category_id'}, inplace=True)
viniferas_table.rename(columns={'cultivar': 'subcategory'}, inplace=True)
viniferas_table.drop(columns=['id'], inplace=True)
viniferas_table = viniferas_table.melt(id_vars=['category_id', 'subcategory', 'variety_id'], var_name='year',
                                       value_name='value')
viniferas_table['year'].replace('2023an', '2023', inplace=True)
viniferas_table['year'] = viniferas_table['year'].astype(int)


In [39]:
def treat_numbers(value):
    # Ensure the input is a string
    value = str(value)
    # Swap comma for dot
    value = value.replace(',', '.')
    # Remove spaces
    value = value.replace(' ', '')

    try:
        # Try converting to float
        return float(value)
    except ValueError:
        # Return original value if conversion fails
        return value

In [40]:
def fill_values_with_nan(data):
    data['value'] = data['value'].apply(treat_numbers)
    data['value'] = pd.to_numeric(data['value'], errors='coerce')
    data.fillna(0, inplace=True)
    return data

In [41]:
viniferas_table = fill_values_with_nan(viniferas_table)


In [42]:
americanas_table['control'] = categorize_data(americanas_table)
americanas_table = filter_data(americanas_table, subcategory_column='cultivar')
americanas_table = map_categories_to_ids(americanas_table, grape_categories, category_column='grapes_categories',
                                         control_column='control', id_column='variety_id')
americanas_table.rename(columns={'control': 'category_id', 'cultivar': 'subcategory'}, inplace=True)
americanas_table.drop(columns=['id'], inplace=True)
americanas_table = americanas_table.melt(id_vars=['category_id', 'subcategory', 'variety_id'], var_name='year',
                                         value_name='value')
americanas_table['year'] = americanas_table['year'].astype(int)

In [43]:
americanas_table = fill_values_with_nan(americanas_table)

In [44]:
mesa_table['control'] = categorize_data(mesa_table)
mesa_table = filter_data(mesa_table, subcategory_column='cultivar')
mesa_table = map_categories_to_ids(mesa_table, grape_categories, category_column='grapes_categories',
                                   control_column='control', id_column='variety_id')
mesa_table.rename(columns={'control': 'category_id', 'cultivar': 'subcategory'}, inplace=True)
mesa_table.drop(columns=['id'], inplace=True)
mesa_table = mesa_table.melt(id_vars=['category_id', 'subcategory', 'variety_id'], var_name='year', value_name='value')
mesa_table['year'] = mesa_table['year'].astype(int)

In [45]:
mesa_table = fill_values_with_nan(mesa_table)

In [46]:
sem_classed_table.rename(columns={'control': 'category_id', 'cultivar': 'subcategory'}, inplace=True)
sem_classed_table.drop(columns=['id'], inplace=True)
sem_classed_table = sem_classed_table.melt(id_vars=['category_id', 'subcategory', 'variety_id'], var_name='year',
                                           value_name='value')
sem_classed_table['year'] = sem_classed_table['year'].astype(int)

In [47]:
sem_classed_table

,category_id,subcategory,variety_id,year,value
0,sc,Sem classificação,3,1970,3675463
1,sc,Sem classificação,3,1971,665425
2,sc,Sem classificação,3,1972,197232
3,sc,Sem classificação,3,1973,491357
4,sc,Sem classificação,3,1974,57307
5,sc,Sem classificação,3,1975,540146
6,sc,Sem classificação,3,1976,24440
7,sc,Sem classificação,3,1977,9743
8,sc,Sem classificação,3,1978,85475
9,sc,Sem classificação,3,1979,2317099


In [48]:
sem_classed_table = fill_values_with_nan(sem_classed_table)

In [49]:
processed_grapes = pd.concat([viniferas_table, americanas_table, mesa_table, sem_classed_table], ignore_index=True)
processed_grapes['year'] = processed_grapes['year'].astype(int)


In [50]:
grape_subcategories = processed_grapes.loc[:, ['category_id', 'subcategory']].drop_duplicates().reset_index(drop=True)
grape_subcategories['subcategory_id'] = grape_subcategories.index

In [51]:
processed_grapes = map_subcategories_to_ids(processed_grapes, grape_subcategories)

Preprocessing imported and exported related datasets

In [52]:
with open('raw_data/opt_05_sub_buttons.json') as f:
    imported_goods = json.load(f)

with open('raw_data/opt_06_sub_buttons.json') as f:
    exported_goods = json.load(f)

In [53]:
imported_goods_varieties = [button['text'] for button in imported_goods['sub_buttons']]
imported_goods_varieties_df = pd.DataFrame(
    columns=['imported_goods'],
    data=imported_goods_varieties
)

In [54]:
exported_goods_varieties = [button['text'] for button in exported_goods['sub_buttons']]
exported_goods_varieties_df = pd.DataFrame(
    columns=['exported_goods'],
    data=exported_goods_varieties
)

In [55]:
all_goods = set(imported_goods_varieties).union(set(exported_goods_varieties))


In [56]:
goods_varieties = pd.DataFrame(
    columns=['goods'],
    data=all_goods
)
goods_varieties['goods_id'] = goods_varieties.index
# Initialize the imported and exported columns with 0
goods_varieties['imported'] = 0
goods_varieties['exported'] = 0

# Set imported column to 1 for imported goods
for good in imported_goods_varieties_df['imported_goods']:
    goods_varieties.loc[goods_varieties['goods'] == good, 'imported'] = 1

# Set exported column to 1 for exported goods
for good in exported_goods_varieties_df['exported_goods']:
    goods_varieties.loc[goods_varieties['goods'] == good, 'exported'] = 1

In [57]:
def process_wine_data(file_path, sep, goods_varieties, search_term, id_col, drop_cols, rename_map, imported=True):
    # Read the CSV file
    df = pd.read_csv(file_path, sep=sep)

    # Extract the goods_id based on the search term
    if imported:
        goods_id = \
        goods_varieties[goods_varieties['goods'].str.contains(search_term) & (goods_varieties['imported'] == 1)][
            'goods_id'].values[0]
    else:
        goods_id = \
        goods_varieties[goods_varieties['goods'].str.contains(search_term) & (goods_varieties['exported'] == 1)][
            'goods_id'].values[0]

    # Add the goods_id to the dataframe
    df[id_col] = goods_id

    # Drop the specified columns
    df.drop(columns=drop_cols, inplace=True)

    # Rename the specified columns
    df.rename(columns=rename_map, inplace=True)

    # Melt the dataframe
    df = df.melt(id_vars=[id_col, 'country'], var_name='year', value_name='value')

    # Fill missing values with NaN
    df = fill_values_with_nan(df)

    # Separate the year and the value type (kg or dollars)
    df['year'] = df['year'].astype(str)
    df['value_type'] = df['year'].apply(lambda x: 'quantity_in_kg' if '.' not in x else 'value_us_dollars')
    df['year'] = df['year'].apply(lambda x: x.split('.')[0])

    # Pivot the dataframe to have separate columns for 'quantity_in_kg' and 'value_us_dollars'
    df = df.pivot_table(index=[id_col, 'country', 'year'], columns='value_type', values='value',
                        aggfunc='first').reset_index()

    return df

In [58]:
imported_wines = process_wine_data('raw_data/ImpVinhos.csv', ';', goods_varieties, 'Vinhos', 'goods_id', ['Id'],
                                   {'País': 'country'})

In [59]:
imported_suco = process_wine_data('raw_data/ImpSuco.csv', ';', goods_varieties, 'Suco', 'goods_id', ['Id'],
                                  {'País': 'country'})

In [60]:
imported_passas = process_wine_data('raw_data/ImpPassas.csv', ';', goods_varieties, 'passas', 'goods_id', ['Id'],
                                    {'País': 'country'})

In [61]:
imported_espumantes = process_wine_data('raw_data/ImpEspumantes.csv', ';', goods_varieties, 'Espumantes', 'goods_id',
                                        ['Id'], {'País': 'country'})

In [62]:
imported_goods = pd.concat([imported_wines, imported_suco, imported_passas, imported_espumantes], ignore_index=True)

In [63]:
exported_wines = process_wine_data('raw_data/ExpVinho.csv', ';', goods_varieties, 'Vinhos', 'goods_id', ['Id'],
                                   {'País': 'country'}, imported=False)

In [64]:
exported_suco = process_wine_data('raw_data/ExpSuco.csv', ';', goods_varieties, 'Suco', 'goods_id', ['Id'],
                                  {'País': 'country'}, imported=False)

In [65]:
goods_varieties[goods_varieties['exported'] == 1]

,goods,goods_id,imported,exported
0,Uvas frescas,0,1,1
1,Espumantes,1,1,1
2,Suco de uva,2,1,1
4,Vinhos de mesa,4,1,1


In [66]:
exported_frescas = process_wine_data('raw_data/ExpUva.csv', ';', goods_varieties, 'frescas', 'goods_id', ['Id'],
                                     {'País': 'country'}, imported=False)


In [67]:
exported_espumantes = process_wine_data('raw_data/ExpEspumantes.csv', ';', goods_varieties, 'Espumantes', 'goods_id',
                                        ['Id'], {'País': 'country'}, imported=False)


In [68]:
exported_goods = pd.concat([exported_wines, exported_suco, exported_frescas, exported_espumantes], ignore_index=True)

In [69]:
#create a directory named transformed_data
import os

if not os.path.exists('transformed_data'):
    os.makedirs('transformed_data')

In [70]:
category_df.drop(columns=['category_id'], inplace=True)
category_df.to_csv('transformed_data/produced_commercialized_categories.csv', index=False)

In [71]:
only_categories_producao.rename(columns={'value': 'quantity_in_l'}, inplace=True)
only_categories_producao.to_csv('transformed_data/produced_categories_with_quantity.csv', index=False)

In [72]:
only_categories_comercio.rename(columns={'value': 'quantity_in_l'}, inplace=True)
only_categories_comercio.to_csv('transformed_data/commercialized_categories_with_quantity.csv', index=False)

In [73]:
subcategories.drop(columns=['subcategory_id'], inplace=True)
subcategories.to_csv('transformed_data/produced_commercialized_subcategories.csv', index=False)

In [74]:
producao_data_subcategory.rename(columns={'value': 'quantity_in_l'}, inplace=True)
producao_data_subcategory.to_csv('transformed_data/produced_subcategories_with_quantity.csv', index=False)

In [75]:
producao_data_subcategory.rename(columns={'value': 'quantity_in_kg'}, inplace=True)
comercio_data_subcategory.to_csv('transformed_data/commercialized_subcategories_with_quantity.csv', index=False)

In [76]:
grapes_varieties_df.to_csv('transformed_data/grape_varieties.csv', index=False)

In [77]:
grape_categories.to_csv('transformed_data/grape_categories.csv', index=False)

In [78]:
grape_subcategories.drop(columns=['subcategory_id'], inplace=True)
grape_subcategories.to_csv('transformed_data/grape_subcategories.csv', index=False)

In [79]:
processed_grapes.rename(columns={'value': 'quantity_in_kg'}, inplace=True)
processed_grapes.to_csv('transformed_data/processed_grapes.csv', index=False)

In [80]:
goods_varieties.drop(columns=['goods_id'], inplace=True)
goods_varieties.to_csv('transformed_data/goods_varieties.csv', index=False)

In [81]:
imported_goods.to_csv('transformed_data/imported_goods.csv', index=False)

In [82]:
exported_goods.to_csv('transformed_data/exported_goods.csv', index=False)

In [83]:
from IPython import get_ipython

ipython = get_ipython()
ipython.system('jupyter nbconvert --to script data-preprocessing.ipynb')

[NbConvertApp] Converting notebook data-preprocessing.ipynb to script
[NbConvertApp] Writing 19962 bytes to data-preprocessing.py
